# Ejercicio 4

In [207]:
import cv2
import numpy as np

## Funciones auxiliares
Dentro de estas funciones, se destaca `motion_kernel`, cuyo resultado es el que se utiliza para obtener la imagen con *Motion Blur*, y `update`, la cual es llamada cada vez que en la GUI se modifique el valor de uno de los *sliders* y aplicará el filtro de Wiener para esos parámetros dados.

In [208]:
def blur_edge(img, d=31):
    h, w  = img.shape[:2]
    img_pad = cv2.copyMakeBorder(img, d, d, d, d, cv2.BORDER_WRAP)
    img_blur = cv2.GaussianBlur(img_pad, (2*d+1, 2*d+1), -1)[d:-d,d:-d]
    y, x = np.indices((h, w))
    dist = np.dstack([x, w-x-1, y, h-y-1]).min(-1)
    w = np.minimum(np.float32(dist)/d, 1.0)
    return img*w + img_blur*(1-w)

In [209]:
def motion_kernel(angle, d, size=65):
    kernel = np.ones((1, d), np.float32)
    cos, sin = np.cos(angle), np.sin(angle)
    A = np.float32([[cos, -sin, 0], [sin, cos, 0]])
    half_size = size // 2

    A[:,2] = (half_size, half_size) - np.dot(A[:,:2], ((d-1)*0.5, 0))
    kernel = cv2.warpAffine(kernel, A, (size, size), flags=cv2.INTER_CUBIC)
    return kernel

In [210]:
def update(img, IMG):
    win = 'Restored with Wiener Filter'

    ang = np.deg2rad( cv2.getTrackbarPos('angle', win) )
    d = cv2.getTrackbarPos('d', win)
    noise = 10**(-0.1*cv2.getTrackbarPos('SNR (db)', win))

    psf = motion_kernel(ang, d, 128)
    cv2.imshow('Distortion Kernel Assumed', psf)

    psf /= psf.sum()
    psf_pad = np.zeros_like(img)
    kh, kw = psf.shape
    psf_pad[:kh, :kw] = psf

    PSF = cv2.dft(psf_pad, flags=cv2.DFT_COMPLEX_OUTPUT, nonzeroRows = kh)
    PSF2 = (PSF**2).sum(-1)

    iPSF = PSF / (PSF2 + noise)[...,np.newaxis]
    RES = cv2.mulSpectrums(IMG, iPSF, 0)

    res = cv2.idft(RES, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT )
    res = np.roll(res, -kh//2, 0)
    res = np.roll(res, -kw//2, 1)
    cv2.imshow(win, res)

## Distorsión de la imagen original
Utilizando el kernel de *Motion Blur* que se muestra en la primer celda a continuación, se distorsiona la imagen nítida de un león.

In [211]:
psf = motion_kernel(np.pi/3, 64, 128)
cv2.imshow('psf', psf)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [212]:
img = cv2.imread('../resources/lion.jpg', cv2.IMREAD_GRAYSCALE)
img = np.float32(img) / img.max()
cv2.imshow('Original Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [213]:
img_motion_blur = cv2.filter2D(img, -1, psf)
img_motion_blur = np.float32(img_motion_blur) / img_motion_blur.max()
img_motion_blur = blur_edge(img_motion_blur, 256)
img_motion_blur = np.float32(img_motion_blur) / img_motion_blur.max()
cv2.imshow('Blurred Image', img_motion_blur)
cv2.waitKey(0)
cv2.destroyAllWindows()
_ = cv2.imwrite('../resources/lion_blurred.jpg', img_motion_blur * 255)

## Reconstrucción con filtro Wiener
Se ejecuta la GUI con la cual se pueden ajustar los parámetros del *Motion Blur* que se supone que fue aplicado, para encontrar el punto de mejor restauración. Dado que se sabe que la distorsión se hizo con ángulo de 60° y `d = 64`, se cargan por defecto esos valores a los sliders, pero puede movérselos para observar la variación en el resultado.

In [ ]:
win = 'Restored with Wiener Filter'
cv2.namedWindow(win)
cv2.namedWindow('Distortion Kernel Assumed', 0)
img_deblur = np.float32(cv2.imread('../resources/lion_blurred.jpg', cv2.IMREAD_GRAYSCALE))
img_deblur /= img_deblur.max()
IMG = cv2.dft(img_deblur, flags=cv2.DFT_COMPLEX_OUTPUT)
cv2.imshow('Original Image', img)
cv2.imshow('Blurred Image', img_motion_blur)

cv2.createTrackbar('angle', win, 60, 180, lambda _ : update(img_deblur, IMG))
cv2.createTrackbar('d', win, 64, 150, lambda _ : update(img_deblur, IMG))
cv2.createTrackbar('SNR (db)', win, 35, 50, lambda _ : update(img_deblur, IMG))
update(img_deblur, IMG)
cv2.waitKey(0)
cv2.destroyAllWindows()